In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import os

In [2]:
load_dotenv()

True

In [3]:
llm = ChatOpenAI(model="gpt-4o")

In [4]:
def get_local_time(input: str = None) -> str:
    from datetime import datetime
    current_time = datetime.now().strftime("%H:%M:%S")
    # print(f"LocalTime tool called, returning: {current_time}")
    return f"Current local time is: {current_time}"

In [5]:
from langchain.agents import AgentExecutor, Tool

In [6]:
tools = [
    Tool(
        name="LocalTime",
        func=get_local_time,
        description="Use this tool to get the current local time."
    )
]

In [7]:
from langchain_core.prompts import PromptTemplate

In [8]:
template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
# When calling a tool, always use the following format:

# ```
# {{
#     "action": "ConcatenateStrings",
#     "action_input": {{
#     "a": "<first_string>",
#     "b": "<second_string>",
#     ...
#     }}
# }}
# ```
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

In [9]:
prompt_template=PromptTemplate.from_template(template)

In [10]:
prompt_template

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\n# When calling a tool, always use the following format:\n\n# ```\n# {{\n#     "action": "ConcatenateStrings",\n#     "action_input": {{\n#     "a": "<first_string>",\n#     "b": "<second_string>",\n#     ...\n#     }}\n# }}\n# ```\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [11]:
from langchain.agents import (
    AgentExecutor,
    create_react_agent,
)

In [12]:
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt_template,
    stop_sequence=True,
)

In [13]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
)


In [14]:
response = agent_executor.invoke({"input": "What time is it?"})



> Entering new AgentExecutor chain...
To find out the current local time, I need to use the LocalTime tool.

Action: LocalTime
Action Input: {}Current local time is: 14:40:40I now know the final answer.

Final Answer: The current local time is 14:40:40.

> Finished chain.


In [15]:
response

{'input': 'What time is it?', 'output': 'The current local time is 14:40:40.'}

In [16]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import tool


/opt/homebrew/Caskroom/miniforge/base/envs/littleSeven/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [17]:
class TestArgs(BaseModel):
    a: str = Field(description="First string")
    b: str = Field(description="Second string")

@tool(args_schema=TestArgs)
def concatenate_strings(a: str, b: str) -> str:
    """Concatenates two strings."""
    print("a", a)
    print("b", b)
    return a + b

In [18]:
@tool()
def get_local_time(input: str = None) -> str:
    """Use this tool to get the current local time."""
    from datetime import datetime
    current_time = datetime.now().strftime("%H:%M:%S")
    # print(f"LocalTime tool called, returning: {current_time}")
    return f"Current local time is: {current_time}"

In [19]:
tools = [
    get_local_time,
    concatenate_strings, 
]

In [20]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder,AIMessagePromptTemplate

In [21]:
system_message=SystemMessagePromptTemplate.from_template(
    """ You are an AI assistant, you need to help the user do right actions and operations. Respond to the human as helpfully and accurately as possible."""
)

history_placeholder=MessagesPlaceholder(variable_name="history",optional=True)

scratchpad_placeholder = MessagesPlaceholder(variable_name="agent_scratchpad", optional=True)

human_message=HumanMessagePromptTemplate.from_template(
    """{input}
    (reminder to respond in a JSON blob no matter what)"""
)

assisstant_message=AIMessagePromptTemplate.from_template(
    "Understood! I will analyze your question type and provide a detailed response."
)

In [22]:
prompt_template=ChatPromptTemplate.from_messages([
    system_message,
    history_placeholder,
    scratchpad_placeholder,
    human_message,
    assisstant_message
])

In [23]:
from langchain.memory import ConversationBufferMemory

In [24]:
memory = ConversationBufferMemory(
    memory_key="history", return_messages=True)

/var/folders/5b/b2jjkdb55_g_qzx3lsl5k59m0000gn/T/ipykernel_24933/1976451665.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [25]:
agent = create_tool_calling_agent(
    llm=llm,  # Language model to use
    tools=tools,  # List of tools available to the agent
    prompt=prompt_template,  # Prompt template to guide the agent's responses
)

In [26]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,  # The agent to execute
    tools=tools,  # List of tools available to the agent
    verbose=True,  # Enable verbose logging
    memory=memory,
    handle_parsing_errors=True,  # Handle parsing errors gracefully
)

In [27]:
response = agent_executor.invoke({"input": "Concatenate 'hello' and 'world'"})



> Entering new AgentExecutor chain...

Invoking: `concatenate_strings` with `{'a': 'hello', 'b': 'world'}`


a hello
b world
helloworld```json
{
  "result": "helloworld"
}
```

> Finished chain.


In [28]:
response = agent_executor.invoke({"input": "What is the time now?"})



> Entering new AgentExecutor chain...

Invoking: `get_local_time` with `{'input': ''}`


Current local time is: 14:40:43
Invoking: `get_local_time` with `{}`


Current local time is: 14:40:43
Invoking: `get_local_time` with `{}`


Current local time is: 14:40:44
Invoking: `get_local_time` with `{}`


Current local time is: 14:40:45```json
{
  "current_local_time": "14:40:45"
}
```

> Finished chain.


In [29]:
response['output']

'```json\n{\n  "current_local_time": "14:40:45"\n}\n```'

In [30]:
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda

In [31]:
memory = ConversationBufferMemory(
    memory_key="history", return_messages=True)

In [32]:
agent = create_tool_calling_agent(
    llm=llm,  # Language model to use
    tools=tools,  # List of tools available to the agent
    prompt=prompt_template,  # Prompt template to guide the agent's responses
)

In [33]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,  # The agent to execute
    tools=tools,  # List of tools available to the agent
    verbose=True,  # Enable verbose logging
    memory=memory,
    handle_parsing_errors=True,  # Handle parsing errors gracefully
)

In [34]:
chain = agent_executor | RunnableLambda(StrOutputParser().parse)

In [35]:
response = chain.invoke({"input": "What is the time now?"})



> Entering new AgentExecutor chain...

Invoking: `get_local_time` with `{}`


Current local time is: 14:40:47```json
{
  "time": "14:40:47"
}
```

> Finished chain.


In [37]:
response

{'input': 'What is the time now?',
 'history': [HumanMessage(content='What is the time now?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='```json\n{\n  "time": "14:40:47"\n}\n```', additional_kwargs={}, response_metadata={})],
 'output': '```json\n{\n  "time": "14:40:47"\n}\n```'}